https://www.bualabs.com/archives/3000/sentiment-classification-imdb-movie-reviews-with-naive-bayes-logistic-regression-nlp-ep-5/

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/

In [7]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [8]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

WindowsPath('C:/Users/user/.fastai/data/imdb_sample')

In [10]:
path = untar_data(URLs.IMDB_SAMPLE)
path

[WindowsPath('C:/Users/user/.fastai/data/imdb_sample/texts.csv')]

In [12]:
path.ls()  #ดูว่ามีไฟล์อะไรบ้าง

(1000, 3)

In [13]:
df = pd.read_csv(path/'texts.csv')  #โหลดไฟล์ csv ขึ้นมา ใส่ Pandas Dataframe
df.shape

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


#มีข้อมูลตัวอย่าง 1000 Record, 3 Column ดูตัวอย่างข้อมูล 5 Record แรก
df.head()

In [17]:
Preprocessing

In [18]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text').split_from_df(col=2).label_from_df(cols=0))
 #เราจะใช้ Fastai TextList ในการ Preprocess ข้อมูล เช่น Tokenization, Numberization, etc.
 #จะได้ Trainint Set, Validation Set ออกมา เรามาดูข้อมูลตัวอย่าง Record แรกใน Validation Set

SyntaxError: invalid syntax (<ipython-input-18-4bd3ee2de335>, line 1)

https://medium.com/ai-in-plain-english/fast-natural-language-processing-101-cf1afefdc449

In [ ]:
movie_reviews.valid.y[0], movie_reviews.valid.x[0]
